In [42]:
# IATI Food Security Data Analysis Notebook
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from dotenv import load_dotenv
import os
import time
from datetime import datetime

In [43]:
load_dotenv()
api_key = os.getenv('IATI_API_KEY')

if not api_key:
    raise ValueError("API key not found. Please make sure it is set in the .env file or update it if necessary.")

In [44]:
# Base URL for the IATI Datastore API
url = "https://api.iatistandard.org/datastore/activity/iati"

In [45]:


# Parameters for the API request
params = {
    'q': 'iati_identifier:"DE-1-202301364-0"',  # Query the specific IATI identifier
    'rows': 1,  # Only need 1 record since it's a specific query
    'start': 0
}

# Headers
headers = {
    'Accept': 'application/xml',  # Accept XML
    'Ocp-Apim-Subscription-Key': api_key
}

all_data = []

def extract_activity_data(activity):
    """Extract data from an activity element into a dictionary."""
    activity_data = {}
    for elem in activity.iter():
        if elem.tag != activity.tag:  
            if elem.text and elem.text.strip():
                activity_data[elem.tag] = elem.text.strip()
            for attr_name, attr_value in elem.attrib.items():
                activity_data[f"{elem.tag}_{attr_name}"] = attr_value
    return activity_data

response = requests.get(url, params=params, headers=headers)
print(f"Status Code: {response.status_code}")

if response.status_code == 200:
    try:
        # Parsing XML
        root = ET.fromstring(response.content)
        activities = root.findall('iati-activity')
        
        # Extracting data
        for activity in activities:
            activity_data = extract_activity_data(activity)
            all_data.append(activity_data)
            
    except ET.ParseError as e:
        print(f"Error parsing XML: {e}")
else:
    print(f"Error: {response.status_code}")

# Converting to DataFrame
df_3640 = pd.DataFrame(all_data)
print(df_3640)


Status Code: 200
    iati-identifier reporting-org_ref reporting-org_secondary-reporter  \
0  DE-1-202301364-0              DE-1                                0   

  reporting-org_type narrative  \
0                 10         0   

  narrative_{http://www.w3.org/XML/1998/namespace}lang description_type  \
0                                                 de                  2   

  participating-org_ref participating-org_role participating-org_type  ...  \
0                  DE-1                      3                     10  ...   

  budget_status budget_type period-start_iso-date period-end_iso-date  \
0             2           2            2023-01-01          2023-12-31   

      value value_value-date transaction-type_code transaction-date_iso-date  \
0  45000.00       2024-04-30                     4                2024-04-30   

  conditions_attached condition_type  
0                   1              1  

[1 rows x 42 columns]


In [46]:
df_3640

,iati-identifier,reporting-org_ref,reporting-org_secondary-reporter,reporting-org_type,narrative,narrative_{http://www.w3.org/XML/1998/namespace}lang,description_type,participating-org_ref,participating-org_role,participating-org_type,...,budget_status,budget_type,period-start_iso-date,period-end_iso-date,value,value_value-date,transaction-type_code,transaction-date_iso-date,conditions_attached,condition_type
0,DE-1-202301364-0,DE-1,0,10,0,de,2,DE-1,3,10,...,2,2,2023-01-01,2023-12-31,45000.00,2024-04-30,4,2024-04-30,1,1


description_type is 2 (is it related to sdg)